<a href="https://colab.research.google.com/github/Sofiiasmm/-_-/blob/main/%D0%A1%D1%83%D1%88%D0%BA%D0%BE_%D0%BB%D0%B0%D0%B13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 🎬 Рекомендаційна система (MovieLens 100k)
# Автор: Софія Сушко
# Методи: МНК, градієнтний спуск, кластеризація
# Користувач: user_id = 18
# ============================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#Завантаження датасету MovieLens 100k
#Якщо файл ще не завантажено, автоматично завантажу
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k/u.data
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k/u.item

#Завантаження даних
columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=columns)

#Назви фільмів
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None)
movies = movies[[0, 1]]
movies.columns = ['item_id', 'title']

#Об’єднання даних
data = pd.merge(ratings, movies, on='item_id')

print("Приклад даних:")
print(data.head())

#Таблиці користувачів-фільмів
user_movie_matrix = data.pivot_table(index='user_id', columns='title', values='rating')

#Метод найменших квадратів (МНК)
print("\n=== Метод найменших квадратів (Linear Regression) ===")

#Користувач
user_id = 18

#Середні рейтинги фільмів
movie_means = data.groupby('title')['rating'].mean()

#Ознакова матриця (dummy для фільмів)
X = pd.get_dummies(data['title'])
y = data['rating']

#Лінійна регресія
model_lr = LinearRegression()
model_lr.fit(X, y)

#Прогноз рейтингу
pred_ratings = model_lr.predict(X)

#Додати до датафрейму
data['predicted_rating'] = pred_ratings

#Фільми, які користувач ще не дивився
user_seen = data[data['user_id'] == user_id]['title'].unique()
recommendations_lr = data[~data['title'].isin(user_seen)].groupby('title')['predicted_rating'].mean().sort_values(ascending=False).head(5)

print("\nРекомендації (МНК):")
print(recommendations_lr)

#Градієнтний спуск (NMF - факторизація матриці)
print("\n=== Градієнтний спуск (NMF) ===")

R = user_movie_matrix.fillna(0)
model_nmf = NMF(n_components=15, init='random', random_state=42, max_iter=300)
user_features = model_nmf.fit_transform(R)
movie_features = model_nmf.components_

R_hat = np.dot(user_features, movie_features)

pred_df = pd.DataFrame(R_hat, index=R.index, columns=R.columns)

recommendations_nmf = pred_df.loc[user_id].sort_values(ascending=False).head(5)

print("\nРекомендації (градієнтний спуск / NMF):")
print(recommendations_nmf)

#Кластеризація (K-Means)
print("\n=== Кластеризація користувачів (K-Means) ===")

#Заповнення пропусків середніми значеннями
R_filled = user_movie_matrix.fillna(user_movie_matrix.mean())

#Масштабування
scaler = StandardScaler()
R_scaled = scaler.fit_transform(R_filled)

#5 кластерів користувачів
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(R_scaled)

#Додавання кластерів до користувачів
user_clusters = pd.DataFrame({'user_id': R.index, 'cluster': clusters})
cluster_id = user_clusters[user_clusters['user_id'] == user_id]['cluster'].values[0]

#Знаходження користувачів із того ж кластера
similar_users = user_clusters[user_clusters['cluster'] == cluster_id]['user_id'].values

#середній рейтинг по кластеру
cluster_mean_ratings = data[data['user_id'].isin(similar_users)].groupby('title')['rating'].mean().sort_values(ascending=False).head(5)

print(f"\nРекомендації (кластеризація, кластер {cluster_id}):")
print(cluster_mean_ratings)

#Підсумок
print("\n✅ Рекомендаційна система побудована успішно!")


Приклад даних:
   user_id  item_id  rating  timestamp                       title
0      196      242       3  881250949                Kolya (1996)
1      186      302       3  891717742    L.A. Confidential (1997)
2       22      377       1  878887116         Heavyweights (1994)
3      244       51       2  880606923  Legends of the Fall (1994)
4      166      346       1  886397596         Jackie Brown (1997)

=== Метод найменших квадратів (Linear Regression) ===
